In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint as DataPoint
from pyspark.mllib.tree import RandomForest
import sys, csv

TEST_DATA = "../data/preprocessed_train_5M.csv"
with open(TEST_DATA) as f:
    reader = csv.reader(f)
    for row in reader:
        print len(row)
        break

83


In [4]:
def ParseData(row):
	data = [float(feature) for feature in row.split(",")]
	return DataPoint(data[0], data[1:20])

#if __name__ == "__main__":
	#conf = SparkConf().setAppName("regressiontest").setMaster("org.apache.spark.deploy.master.Master")
	#sc = SparkContext(conf=conf)
data = sc.textFile(TEST_DATA).map(ParseData)
train, test = data.randomSplit([0.6, 0.4], seed = 11L)

RF = RandomForest.trainClassifier(train, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=10, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=5, maxBins=32)

predictions = RF.predict(test.map(lambda x: x.features))
labelsAndPredictions = test.map(lambda lp: lp.label).zip(predictions)

testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(test.count())
print testErr

0.248109451567


In [5]:
RF.clearThreshold()
scores = RF.predict(test.map(lambda x: x.features))
labelsAndScores = test.map(lambda lp: lp.label).zip(scores)
print labelsAndScores

AttributeError: 'RandomForestModel' object has no attribute 'clearThreshold'